In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Border, Side, Font
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import files
import io

In [ ]:
print("Upload your Deals CSV file:")
deals_uploaded = files.upload()

print("Upload your Headcount CSV file:")
headcount_uploaded = files.upload()

print("Upload your Excel Template:")
template_uploaded = files.upload()

Upload your Deals CSV file:


Saving Outsourcing Vendor Deals Schema.csv to Outsourcing Vendor Deals Schema (1).csv
Upload your Headcount CSV file:


Saving Headcount.csv to Headcount (1).csv
Upload your Excel Template:


Saving Deals_Formatters.xlsx to Deals_Formatters (1).xlsx


In [ ]:
deals_df = pd.read_csv(io.BytesIO(deals_uploaded[list(deals_uploaded.keys())[0]]))
headcount_df = pd.read_csv(io.BytesIO(headcount_uploaded[list(headcount_uploaded.keys())[0]]))
template_bytes = template_uploaded[list(template_uploaded.keys())[0]]

<ipython-input-6-54b9a3958c2e>:1: DtypeWarning: Columns (20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  deals_df = pd.read_csv(io.BytesIO(deals_uploaded[list(deals_uploaded.keys())[0]]))


In [ ]:
def clean_and_format_data(deals: pd.DataFrame, headcount: pd.DataFrame):
    required_columns = ['Description', 'Deal Start Date', 'Deal End Date', 'Headcount',
                        'Linkedin Link', 'Gmail Id', 'Client Name', 'Provider Name',
                        'Provider MSA', 'Client MSA']
    missing_cols = [col for col in required_columns if col not in deals.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in Deals file: {', '.join(missing_cols)}")

    deals['Description'] = deals['Description'].str.replace('[-=]', '', regex=True)
    deals = deals[~deals['Description'].str.contains("#NAME?", na=False)]

    deals['Deal Start Date'] = pd.to_datetime(deals['Deal Start Date'], errors='coerce')
    deals['Deal End Date'] = pd.to_datetime(deals['Deal End Date'], errors='coerce')

    deals['Formatted Start Date'] = deals['Deal Start Date'].apply(
        lambda x: f"Q{int((x.month + 2) / 3)} {x.year}" if pd.notnull(x) else '-')
    deals['Formatted End Date'] = deals['Deal End Date'].apply(
        lambda x: f"Q{int((x.month + 2) / 3)} {x.year}" if pd.notnull(x) else '-')

    headcount_mapping = headcount.set_index('Main')['Range']
    deals['Headcount Range'] = deals['Headcount'].map(headcount_mapping).fillna('-')

    deals['Linkedin Link_clean'] = deals['Linkedin Link'].apply(lambda x: x if 'linkedin.com' in str(x) else None)
    deals['Gmail Id_clean'] = deals['Gmail Id'].apply(lambda x: x if '@gmail.com' in str(x) else None)

    deals['LinkedIn_URL_CVID'] = deals.apply(
        lambda row: row['Linkedin Link_clean'] or row['Gmail Id_clean'], axis=1).fillna('Secondary Research')

    deals['Duplication check'] = (
        deals['Client Name'] + deals['Provider Name'] + deals['Description'] +
        deals['Provider MSA'] + deals['Formatted Start Date']
    )
    deals = deals.drop_duplicates(subset='Duplication check')
    deals.fillna('-', inplace=True)

    internal = deals[[
        'Deal Id', 'Client Name', 'Draup Verticals', 'Provider Name', 'Client MSA', 'Provider MSA',
        'Headcount Range', 'Description', 'Formatted Start Date', 'Business Function',
        'Functional Workload', 'Digital Product', 'Skills', 'Digital Technology Evidence',
        'Headcount', 'Deal Start Date'
    ]]

    client = deals[[
        'Deal Id', 'Client Name', 'Draup Verticals', 'Provider Name', 'Provider MSA', 'Description',
        'Formatted Start Date', 'Functional Workload', 'Digital Product', 'Skills',
        'Digital Technology Evidence'
    ]]

    zinnov = deals[[
        'Deal Id', 'Client Name', 'Draup Verticals', 'Provider Name', 'Client MSA', 'Provider MSA',
        'Headcount Range', 'Description', 'Formatted Start Date', 'Formatted End Date',
        'Business Function', 'Functional Workload', 'Digital Product', 'Skills',
        'Digital Technology Evidence', 'LinkedIn_URL_CVID', 'Headcount', 'Client Subsidiary'
    ]]

    return internal, client, zinnov

In [ ]:
internal_df, client_df, zinnov_df = clean_and_format_data(deals_df, headcount_df)

<ipython-input-7-31d43e55d777>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  deals.fillna('-', inplace=True)
<ipython-input-7-31d43e55d777>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  deals.fillna('-', inplace=True)


In [ ]:
class DashboardFormatter:
    def __init__(self, template_bytes, title=None):
        self.workbook = load_workbook(io.BytesIO(template_bytes))
        self.title = title

    def _set_title(self, sheet, cell_address):
        sheet[cell_address] = self.title
        sheet[cell_address].font = Font(name="Calibri", size=10, italic=True)

    def _paste_dataframe(self, sheet, df, start_row, start_col):
        font_style = Font(name="Calibri", size=10)
        for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), start=start_row):
            for c_idx, val in enumerate(row, start=start_col):
                cell = sheet.cell(row=r_idx, column=c_idx, value=val)
                cell.font = font_style

    def _align_columns(self, sheet, center_cols, left_cols, start_row):
        for col in center_cols:
            for cell in sheet[col]:
                if cell.row >= start_row:
                    cell.alignment = Alignment(horizontal="center", vertical="center")
        for col in left_cols:
            for cell in sheet[col]:
                if cell.row >= start_row:
                    cell.alignment = Alignment(horizontal="left")

    def _adjust_column_width(self, sheet, columns, padding=0):
        for col in columns:
            max_len = max((len(str(cell.value)) for cell in sheet[col] if cell.value), default=0)
            sheet.column_dimensions[col].width = max_len + padding

    def _set_borders(self, sheet, start_row, start_col, num_rows, num_cols):
        thin = Side(style='thin')
        medium = Side(style='medium')
        for r in range(start_row, start_row + num_rows):
            for c in range(start_col, start_col + num_cols):
                cell = sheet.cell(row=r, column=c)
                cell.border = Border(left=thin, right=thin, top=thin, bottom=thin)
            sheet.cell(row=r, column=start_col).border = Border(left=medium)
            sheet.cell(row=r, column=start_col+num_cols-1).border = Border(right=medium)
        for c in range(start_col, start_col + num_cols):
            sheet.cell(row=start_row+num_rows-1, column=c).border = Border(bottom=medium)

    def _center_align_dash_cells(self, sheet):
        for row in sheet.iter_rows():
            for cell in row:
                if cell.value == "-":
                    cell.alignment = Alignment(horizontal="center")

    def _process_sheet(self, name, df, title_cell, start_row, start_col, center_cols, left_cols, width_cols):
        sheet = self.workbook[name]
        self._set_title(sheet, title_cell)
        self._paste_dataframe(sheet, df, start_row, start_col)
        self._align_columns(sheet, center_cols, left_cols, start_row)
        self._adjust_column_width(sheet, width_cols, padding=2)
        self._set_borders(sheet, start_row, start_col, len(df), len(df.columns))
        self._center_align_dash_cells(sheet)

    def internal_deals(self, df):
        self._process_sheet("Outsourcing Dashboard", df, "C3", 6, 3,
            [get_column_letter(i) for i in range(3, 16)], ['E','J','N','O','P'], ['L'])

    def client_deals(self, df):
        self._process_sheet("deals", df, "C3", 6, 3,
            [get_column_letter(i) for i in range(3, 13)], ['E','H','K','L','M'], ['L'])

    def zinnov_deals(self, df):
        self._process_sheet("zinnov", df, "E3", 6, 7,
            [get_column_letter(i) for i in range(7, 22)], ['I','N','S','T','U','V'], ['J'])

    def save(self, filename):
        self.workbook.save(filename)
        return filename

In [ ]:
formatter = DashboardFormatter(template_bytes, title="Requested Accounts")
formatter.internal_deals(internal_df)  # You can change to client_df or zinnov_df
output_path = formatter.save("deals_internal_formatted.xlsx")

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>